# Drew K's final Capstone Assignment - ASDOOR Limited and David's big move to Toronto

### A. Introduction - Outlining problem or idea summary <a name="introduction"></a>

A businessman (David) has been asked by the company that he works for (ASDOOR Limited), to move to Toronto Canada to setup a new warehouse where ASDOOR would like begin operations from, in order to supply the city of Toronto in a more timely manner, and also make use of Toronto's more lenient tax programmes, and cheap rental rates. As part of this move David will need to move his entire family (David and his wife, and 2 children) to the city of Toronto, which will be a significant move for this family. As such David will need to carefully consider where he would like the family to settle in Toronto to ensure he is close to venues and attractions that the family would like to regularly visit. Also, David has a great responsibility for ASDOOR Limited to ensure he chooses a great location for the ASDOOR factory / warehouse to set up operations.

Thus in my assignment I will study the location data available for Toronto Canada, to determine appropriate locations that the ASDOOR factory/warehouse should locate itself, and similarly where David's family should buy/rent a home in order to have an enjoyable living experience in Toronto.

### B. Describe the data that you will be using to solve the problem or execute your idea

I will be using the postal codes of Canada dataset scraped from the web (wikipedia) to determine the boroughs/neighbourhoods of Canada. I will then begin examining the data, stripping down the data to look at Toronto city only. Once I have determined the Toronto postcodes/neighbourhoods I will then analyse;
1. By using Foursquare data, I will determine the number of venues in each neighbourhood
2. Using Foursquare data, I will then determine the unique categories of venues in those neighbourhoods, and determine the top 5 categories of venue in each neighbourhood
3. I will then cluster those neighbourhoods to determine similar cluster types (similar neighbourhoods)
4. Once I have hopefully found distinct clusters, I will investigate each cluster type to determine where is appropriate to setup a warehouse for ASDOOR Limited and where is possibly appropriate for David and his family to settle.
5. Finally, once I believe I have possibly determined the appropriate locations for ASDOOR Limited to develop its warehouse, and Davids family to rent/buy their house, I will specifically investigate those 2 locations for a more detailed analysis of venues so that I am comfortable that the areas are appropriate for the requirements (those being 1. Develop warehouse in appropriate location, and 2. Settle Davids family in a nice postcode that will resonate with the family.

Note: The data that is being scraped is found here: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### C. Methodology to determine core requirements 1. Where should ASDOOR locate its Factory / Warehouse and 2. Where should Davids family choose to live

##### C1. Load Dependencies

In [114]:
#Core dependencies
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [115]:
#Need this patch to enable the remainder dependencies to load correctly. Unsure why was necessary - yet enabled to work / execture
!pip install folium

In [110]:
#Remainder dependencies
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


##### C2: Get the coordinates for Canada neighbourhoods and then drill down further to only look at Toronto neighbourhoods

In [116]:
#Prepare data and determine number of rows
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head(10)
#ignore where borough is not assigned
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#use lambda function to join where same postcode exists for cases where different neighbourhood
df=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
#where borough exists and neighbourhood does not exist = set nei. == bor.
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']
#notebook clean
print ("no. rows= ",df.shape[0])

no. rows=  103


In [118]:
geo_coords=pd.read_csv("https://cocl.us/Geospatial_data")
df['Latitude']=geo_coords['Latitude'].values
df['Longitude']=geo_coords['Longitude'].values
df

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Kennedy Park,East Birchmount Park,Ionview",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476


In [119]:
#Drill down to Toronto only postcodes / neighbourhoods
#new df only borough, toronto

toro_df = df[ df.Borough.str.contains('Toronto') ]
toro_df.reset_index(inplace=True)
toro_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Summerhill East,Moore Park",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Rathnelly,South Hill,Forest Hill SE,...",43.686412,-79.400049


##### C3: Get Toronto Geographical coordinates and produce map of Toronto with all neighbourhoods (not yet clustered)

In [120]:
#Explore the neighbourhoods in Toronto. 
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada is 43.653963, -79.387207.


In [121]:
# create map of Toronto
map_toro = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toro_df['Latitude'], toro_df['Longitude'], toro_df['Borough'], toro_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toro)  
    
map_toro

##### C4: Using Foursquare ( https://foursquare.com ), determine most common nearby venues (shop type etc) of neighbourhoods / locations, to enable clustering of neighbourhood by venue (shope type etc) type

In [122]:
#Define Foursquare
CLIENT_ID = 'S2QY0JK2O15HKR3OERXKMOFGFTH5IQHIIWLMASD4T4ZPDPRA' # your Foursquare ID
CLIENT_SECRET = 'JEAL4X5W4HH1KENWSLWG5PZLTPP13EOF5R1XLEVEQMR01ZF0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S2QY0JK2O15HKR3OERXKMOFGFTH5IQHIIWLMASD4T4ZPDPRA
CLIENT_SECRET:JEAL4X5W4HH1KENWSLWG5PZLTPP13EOF5R1XLEVEQMR01ZF0


In [123]:
#Import dependencies for investigation
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

##### C4(a) - Firstly investigate Victoria Hotel to see if appropriate area for either Warehouse or Home to live? 

In [124]:
#Begin investigation of an area that may be likely to be good for ASDOOR
#chose Victoria Hotel, Downtown Toronto (index[21]) to explore, set up attributes

neighbourhood_latitude = toro_df.loc[21, 'Latitude']
neighbourhood_longitude = toro_df.loc[21, 'Longitude']
neighbourhood_name = toro_df.loc[21, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Commerce Court,Victoria Hotel are 43.6481985, -79.37981690000001.


In [125]:
#Top 100 venues around Victoria Hotel
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=S2QY0JK2O15HKR3OERXKMOFGFTH5IQHIIWLMASD4T4ZPDPRA&client_secret=JEAL4X5W4HH1KENWSLWG5PZLTPP13EOF5R1XLEVEQMR01ZF0&v=20180605&ll=43.6481985,-79.37981690000001&radius=500&limit=100'

In [126]:
#Get and examine results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7a7d38c530930037db28e8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 189,
  'suggestedBounds': {'ne': {'lat': 43.6526985045, 'lng': -79.37360952665502},
   'sw': {'lat': 43.6436984955, 'lng': -79.386024273345}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05df964a52059f620e3',
       'name': 'Canoe',
       'location': {'address': '66 Wellington St West',
        'crossStreet': 'at Bay Street',
        'lat': 43.647452066183476,
        'lng': -79.38132001815676,
        'labeledLatLngs': [{

In [127]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [128]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Canoe,Restaurant,43.647452,-79.381320
1,Equinox Bay Street,Gym,43.648100,-79.379989
2,Mos Mos Coffee,Café,43.648159,-79.378745
3,Walrus Pub & Beer Hall,Pub,43.647375,-79.379515
4,Pilot Coffee Roasters,Coffee Shop,43.648835,-79.380936
5,King Taps,Gastropub,43.648476,-79.382058
6,Adelaide Club Toronto,Gym / Fitness Center,43.649279,-79.381921
7,Maman,Café,43.648309,-79.382253
8,Hockey Hall Of Fame (Hockey Hall of Fame),Museum,43.646974,-79.377323
9,Cactus Club Cafe,American Restaurant,43.649552,-79.381671


In [129]:
#Number of venues returned by Foursquare (expect 100)
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<b>Summary of Victoria Hotel area:</b> Appears to be very inner city orientated - restaurants / bars / cafes. Initial proposition that this area does not "feel" appropriate for either a "young" family (with children), nor does it appear appropriate for a warehouse (too dense with people looking for food places, hence will impact trucking logistics ability to easily access a warehouse in that area. I.e. trucks won't be able to easily access this area).

##### C4(b) Investigate all neighbour hoods in Toronto, to analyse their common Venues and determine clusters.

In [130]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [131]:
#Toronto Venues nearby to the neighbourhoods

toronto_venues = getNearbyVenues(names=toro_df['Neighbourhood'],
                                   latitudes=toro_df['Latitude'],
                                   longitudes=toro_df['Longitude']
                                  )


The Beaches
Riverdale,The Danforth West
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East,Moore Park
Deer Park,Rathnelly,South Hill,Forest Hill SE,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Garden District,Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide,Richmond,King
Union Station,Harbourfront East,Toronto Islands
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill West,Forest Hill North
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Grange Park,Chinatown,Kensington Market
South Niagara,Railway Lands,King and Spadina,Island airport,CN Tower,Bathurst Quay,Harbourfront West
Stn A PO Boxes 25 The Esplanade
Underground city,First Canadian Place
Christie
Dovercourt Village,Dufferin
Trinity,Little Portugal
Parkdale Village,Exhibition Place,Brockton
High Park,The Junction South
Roncesvalles,Parkda

In [132]:
#Size of dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1719, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [133]:
#Venues returned for each Neighbourhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,Richmond,King",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Bay Street,86,86,86,86,86,86
Christie,16,16,16,16,16,16
Church and Wellesley,88,88,88,88,88,88
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38


In [134]:
#Unique categories for all neighbourhoods
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


In [135]:
#Analysing each neighbourhood - to provide the venues
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Riverdale,The Danforth West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Riverdale,The Danforth West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Riverdale,The Danforth West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Riverdale,The Danforth West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Riverdale,The Danforth West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [136]:
#New dataframe shape - expect to see 1719 rows and hidden columns (columns will total 238 however)
toronto_onehot.shape

(1719, 238)

In [137]:
#Neighbourhoods are then grouped together, with the mean of the frequency of the Category (shop type)
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,Richmond,King",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.022727,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011628,0.000000,...,0.000000,0.000000,0.00,0.011628,0.000000,0.000000,0.011628,0.000000,0.000000,0.011628
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,0.000000,...,0.000000,0.000000,0.00,0.000000,0.011364,0.011364,0.000000,0.000000,0.011364,0.011364
7,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.026316,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [138]:
#Print each neighbourhood with its 10 most common venue type
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,Richmond,King----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2                  Bar  0.04
3           Steakhouse  0.04
4         Burger Joint  0.03
5           Restaurant  0.03
6       Cosmetics Shop  0.03
7  American Restaurant  0.03
8      Thai Restaurant  0.03
9                Hotel  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3          Steakhouse  0.04
4  Seafood Restaurant  0.04
5      Farmers Market  0.04
6         Cheese Shop  0.04
7              Bakery  0.04
8                Café  0.04
9       Shopping Mall  0.02


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.05
2         Auto Workshop  0.05
3                  Park  0.05
4            Comic Shop  0.05
5            Restaurant  0.05
6        Farmers Market  0.05
7  Fast Food Restaur

In [139]:
#Placeing the above in a pandas dataframe, and sorts venues by descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [140]:
#Now top 10 venues per neighbourhood. I want the top 10 to get a good feel of the granularity of the neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,Richmond,King",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,Cosmetics Shop,Restaurant,American Restaurant,Hotel,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Cheese Shop,Seafood Restaurant,Beer Bar,Steakhouse,Café,Irish Pub
2,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Comic Shop,Gym / Fitness Center
3,"Cabbagetown,St. James Town",Coffee Shop,Market,Bakery,Café,Italian Restaurant,Pub,Pizza Place,Restaurant,Breakfast Spot,Caribbean Restaurant
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Sandwich Place,Burger Joint,Ice Cream Shop,Spa,Chinese Restaurant,Bar
5,Christie,Grocery Store,Café,Park,Diner,Nightclub,Convenience Store,Baby Store,Italian Restaurant,Coffee Shop,Athletics & Sports
6,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Burger Joint,Café,Pub,Bubble Tea Shop,Pizza Place
7,"Commerce Court,Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Seafood Restaurant,Steakhouse,Deli / Bodega,Gym
8,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Italian Restaurant,Restaurant,Café,Sushi Restaurant,Thai Restaurant
9,Davisville North,Sandwich Place,Gym,Breakfast Spot,Park,Restaurant,Hotel,Food & Drink Shop,Clothing Store,Dance Studio,Doner Restaurant


<b>Observations of above:</b>On initial examination of the above, an observation is that there appears to be alot of locations that are heavily coffee/restaurant orientated, and I presume these will be in the heavily built up / business areas. I believe that places like "The Beaches" (number 33) should not be within the cluster with the restaurant/bar/cafe areas, as it appears quite distinct from a quick initial overview of its venues (great outdoors (presume parks etc), dog run, trail). Similarly number 2 and 27 (Business Reply  Mail Processing Centre and South Niagara respectfully), appear to be more transport / logistic / market hubs (railway/airport/airport lounge/harbour/garden centre/farmers markets are common venue types), hence I would not expect these locations to be clustered in with the restaurant/cafe neighbourhoods.

##### C5: Now for Clustering the Neighbourhoods to see which cluster with which (i.e. have common venue types)

<p><p> Here we use K-Means clustering to determine the clusters

In [141]:
# set number of clusters. I am hoping that by using more clusters I will be able to differentiate more successfully.
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:8] 

array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [142]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toro_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,6,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Pub,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Brewery,Bubble Tea Shop,Café
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Liquor Store,Ice Cream Shop,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Cheese Shop,Stationery Store,Fish Market,Bookstore,Brewery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [143]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### C6: Examination of the 8 Clusters

###### Cluster 1

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Riverdale,The Danforth West",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Yoga Studio,Brewery,Bubble Tea Shop,Café
2,"The Beaches West,India Bazaar",0,Park,Sandwich Place,Liquor Store,Ice Cream Shop,Burger Joint,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Food & Drink Shop
3,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Cheese Shop,Stationery Store,Fish Market,Bookstore,Brewery
5,Davisville North,0,Sandwich Place,Gym,Breakfast Spot,Park,Restaurant,Hotel,Food & Drink Shop,Clothing Store,Dance Studio,Doner Restaurant
6,North Toronto West,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Italian Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park,Health & Beauty Service,Bagel Shop
7,Davisville,0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Italian Restaurant,Restaurant,Café,Sushi Restaurant,Thai Restaurant
9,"Deer Park,Rathnelly,South Hill,Forest Hill SE,...",0,Pub,Coffee Shop,Pizza Place,American Restaurant,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,Supermarket,Restaurant
11,"Cabbagetown,St. James Town",0,Coffee Shop,Market,Bakery,Café,Italian Restaurant,Pub,Pizza Place,Restaurant,Breakfast Spot,Caribbean Restaurant
12,Church and Wellesley,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Burger Joint,Café,Pub,Bubble Tea Shop,Pizza Place
13,"Harbourfront,Regent Park",0,Coffee Shop,Pub,Bakery,Park,Café,Mexican Restaurant,Restaurant,Gym / Fitness Center,Breakfast Spot,Theater


###### Cluster 2

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Summerhill East,Moore Park",1,Playground,Tennis Court,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


###### Cluster 3

In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,2,Garden,Pool,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


###### Cluster 4

In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,3,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


###### Cluster 5

In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,4,Park,Playground,Trail,Building,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


###### Cluster 6

In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"Forest Hill West,Forest Hill North",5,Trail,Jewelry Store,Sushi Restaurant,Park,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Yoga Studio


###### Cluster 7

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,6,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Pub,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


###### Cluster 8

In [151]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,"South Niagara,Railway Lands,King and Spadina,I...",7,Airport Service,Airport Terminal,Boutique,Coffee Shop,Boat or Ferry,Sculpture Garden,Harbor / Marina,Bar,Airport Lounge,Airport Gate


### D: Results section

<b>Summary of above outputs/results:</b>
<li>Cluster 1 is clearly a cluster dominated by coffee shops and restaurants. It is a very large cluster of circa 31 neighbourhoods. This I was expecting to see, however one strong/significant outlier exists - that being number 37, Business Reply Mail Processing Centre 969 Eastern. I was expecting to see number 37 in its own cluster, as it appears to have quite distinct venues from the others in Cluster 1. 
<li>Clusters 2,3,4,5,6 and 7 appear to have similar "family/neighbourhood" type venues, inter-dispersed with restaurants, and although I would hope to see these clustered together, there is enough dis-similarity between the venue types that it makes sense they are not clustered together. 
    <li> Cluster 8 is a logical cluster of its own. It appears to be a large Airport hub which makes sense that it should be clustered on its own
        <p>
           

### E: Observations section

 <b>Observations / Interpretation relating to this assignment:</b><p>
            In the above clustering I was hoping to see as distinct clusters;
            <li>1. Business Reply Mail Processing Centre 969 Eastern
                <li>2. A large inner city coffee/restaurant precinct
                    <li>3. South Niagara Airport precinct
                        <li>4. A cluster of "residential/neighbourhood" family orientated areas. <p><p>
                            This has not been the case however. Although the Airport hub is split out on its own, the Business Reply Mail Processing Centre was listed amongst the restaurants/coffee shop hub and was not separate out on its own as I was expecting. Also the residential / family neighbourhood areas I expected to be clustered together were not clustered together, rather were clustered individually. This makes sense however, as I forced the K number as 8, which is quite a high number.<p><p>I believe an issue maybe that I have added in too many clusters. Perhaps if 4 or 5 were used there maybe better clusters (i.e. enabling all family neighbourhood areas to cluster together). <p>
To reinforce, it does surprise me that the Business Reply Mail Processing Centre was not split out on its own, and this was the location that I was hoping would appear attractive to David to locate the new ASDOOR Limited factory. I initially expected this area to be a good location due to its "industrial" type venues surrounding e.g. Light Rail, Garden Centre and Park, that was observed when I reviewed the top 10 venues for each neighbourhood. Intuitively, light rail to me is an "industrial" term, and evokes the feeling a train line being passed through a more vacant and area / region - however my interepretation maybe different to that of Foursquares interpretation, and light rail may run through very dense areas in Toronto (as it appears to). So it is based on the "venue" interpretation by Foursquare. To me, it appears that the Business Reply Mail neighbourhood has sufficient distance from the average cluster centroid for that Cluster 1, however obviously the algorithm did not consider that neighbourhood to be as distant from the centroid for cluster 1 that I would have expected. Hence it is all about interpretation.                  
                      
However - a key output from this is that the neighbourhoods that initially appeared attractive to me, such as 1. The Beaches (Cluster 7), 2. Rosedale (Cluster 5), 3. Lawrence Park (Cluster 4) and 4. Roselawn (Cluster 3) all are separated out as more family/neighbourhood orientated areas.<p><p>
    I would expect David to choose to live in an area such as "The Beaches", Cluster 7, as it has a clear "Neighbourhood" venue/area as well as "The great outdoors" and a "Trail" as well as "dog run" for their lovely big Weimaraner dog.<p><p>
        As outlined above, a recommendation would be to try to run this same process with fewer centroids (number of K's), to determine if the clustering works better. In a prior module, I do recall there being an issue when you increase the number of "K" too highly, and this appears to be the case here.

### F: Conclusion

In conclusion, David should try to settle his own family in "The Beaches" (Cluster 7), as this appears to have a number of venues which resonate with his family who love the outdoors so much, and also because of the dog they will be bringing with them, which will be able to be walked at the local "dog run" by the son of David, Matthew. David should also search for a location with an industrial / warehousing theme to it, which to me appears to be the "Business Reply Mail Processing Centre" neighbourhood, yet the K-Means clustering algorithm clealy did not see this as a differing location/area to that of the city centre coffee shops / restaurant neighbourhoods. Thus the clustering that has occured in this project has not been ideal, and additional clarrification of clusters needs to be obtained (but reducing numbers quite likely), and also by ensuring the cluster are sufficiently "alike". Clearly there is a differing interpretation of Clusters between myself (Drew), and the K-Means interpretation of the Foursquare Venue dataset..... It  computer interpretation of "alike" venues, is not that advanced yet....<p><p>
    Dedicated to Hanh. <p><p>Goodnight!